In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle
import datetime
import numpy as np

Specifying all the focal brands

In [2]:
focal_brands = ['Sephora',
 'ULTA Beauty',
 'Olive Garden',
 'The Cheesecake Factory',
 'Target',
 'Walmart',
 'Anthropologie',
 "Victoria's Secret"]

Reading the social brands catalog to get visits later for each store

In [3]:
brands_visits = pd.read_csv('../data/revision_visits_revenue_2019.csv')
brands_visits['brand_standard'] = brands_visits['brand'].apply(lambda x: x.strip().lower()) # For comparison with catalog.tsv
brands_visits['date'] = brands_visits['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
brands_visits = brands_visits.rename(columns={'brand': 'brand_visitation'})
brands_visits.head()

,date,PLACEKEY,visits_by_day,spend_by_day,brand_visitation,lat,lon,brand_standard
0,2019-06-01,zzw-222@62j-sgj-q2k,5,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
1,2019-06-02,zzw-222@62j-sgj-q2k,1,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
2,2019-06-03,zzw-222@62j-sgj-q2k,6,859.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
3,2019-06-04,zzw-222@62j-sgj-q2k,6,30.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
4,2019-06-05,zzw-222@62j-sgj-q2k,8,193.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness


Reading Travel Time

In [4]:
with open('../data/travel_time.pkl', 'rb') as file:
    travel_time_dict = pickle.load(file)
    
travel_time_keys = list(travel_time_dict.keys())
from_keys = [key[0] for key in travel_time_keys]
to_keys = [key[1] for key in travel_time_keys]
time_minutes = list(travel_time_dict.values())
time_minutes = [int(time_inst.split(' ')[0]) for time_inst in time_minutes]

travel_time = pd.DataFrame({'From_PLACEKEY': from_keys, 'To_PLACEKEY': to_keys, 'Time_mins': time_minutes})
travel_time.head()

,From_PLACEKEY,To_PLACEKEY,Time_mins
0,zzw-224@62k-p96-s5z,zzw-223@62k-ns4-pn5,20
1,zzw-224@62k-p96-s5z,zzy-222@62k-pd8-975,20
2,zzw-224@62k-p96-s5z,237-222@62k-p8v-z4v,16
3,zzw-224@62k-p96-s5z,222-222@62k-p8v-2p9,12
4,zzw-224@62k-p96-s5z,229-222@62k-p76-d9z,14


Reading the statistics of the specific focal brand using the results of Part 2

In [5]:
brand = focal_brands[0]
focal_brand_path = os.path.join('../part2_r_statistics', brand)
focal_brand_path

'../part2_r_statistics/Sephora'

Reading all the neighboring brands results for the selected focal brand

In [6]:
file_list = os.listdir(focal_brand_path)
result_file_list = [file for file in file_list if file.find('_result') != -1]

In [7]:
result_df = pd.DataFrame()

for result_file in result_file_list:
    result_file_path = os.path.join(focal_brand_path, result_file)
    tmp_res_df = pd.read_csv(result_file_path, skiprows=1, float_precision="round_trip")
    # Removing all the records for ols and m_olsExp
    if 'filename' in tmp_res_df['filename'].tolist(): #Wrong input
        continue
    result_df = pd.concat([result_df, tmp_res_df], ignore_index=True)

In [8]:
result_df = result_df[~result_df['type'].isin(['ols', 'm_olsExp'])].reset_index(drop=True)
result_df = result_df.replace('FALSE', False).replace('False', False)
# result_df.iloc[:,3:] = result_df.iloc[:,3:].astype('float64', copy=True)

In [9]:
result_df

,tmp,filename,type,X_Estimate,X_Std. Error,X_t value,X_Pr(>|t|),IV_firststage_reviews_tw_Estimate,IV_firststage_reviews_tw_Std. Error,IV_firststage_reviews_tw_t value,...,HausWutest_Df,HausWutest_F,HausWutest_Pr(>F),Sargan_result_rsq,Sargan_result_adjrsq,Sargan_pvalue_rsq,Sargan_pvalue_adjrsq,num_sig_variables,Y_r.squared,Y_adj.r.squared
0,tmp,Brooks Brothers,fe_reviews_reviews,0.069505,0.019034,3.651686,2.663556e-04,-0.000100211019005055,0.000190635514929389,-0.52566815287368,...,-1,2.34354613255589,0.125940823615684,4.23449349278573,-353.500987164209,0.120362561387644,1,2,0.671032,0.625286
1,tmp,Brooks Brothers,fe_reviews_visits,0.068299,0.003906,17.487215,2.233359e-64,False,False,False,...,-1,1.9329373522152,0.164572372068733,1.21490380085773e-28,-355.731304347826,1,1,1,0.671424,0.625732
2,tmp,Brooks Brothers,fe_exp_reviews_reviews,1.541672,0.234746,6.567409,6.312742e-11,0.377923078481934,0.318135845031044,1.18792988713691,...,-1,14.7189627428136,0.000128155117934805,0.74492433429816,-357.468891520661,0.689035719250415,1,2,0.727018,0.689058
3,tmp,Brooks Brothers,fe_exp_reviews_visits,1.064166,0.030081,35.376507,3.701041e-219,False,False,False,...,-1,37.0578552273081,1.34043757762807e-09,2.74781634341386e-28,-355.731304347826,1,1,1,0.735753,0.699007
4,tmp,Torrid,fe_reviews_reviews,0.060262,0.021028,2.865750,4.187351e-03,0.00498750665721031,0.00258175309132841,1.93182945106654,...,-1,14.496389999924,0.000143060081773808,3.21982428292259,-346.092335329809,0.199905176680324,1,0,0.109297,0.019754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,tmp,REI,fe_exp_reviews_visits,17.982587,11.619486,1.547623,1.218157e-01,False,False,False,...,-1,159.888003382137,9.11891570094413e-36,3.13423717663725e-28,-347.234455958549,1,1,0,0.011222,-0.091762
1476,tmp,B,fe_reviews_reviews,1.637189,1.288917,1.270205,2.041267e-01,-0.000207663876637634,0.000482982056227962,-0.429961887734437,...,-1,4.07898337207887,0.0435235342635979,9.96297734472914,-343.774750838493,0.00686383694371739,1,1,0.044036,-0.075134
1477,tmp,B,fe_reviews_visits,4.949395,1.881332,2.630792,8.569535e-03,False,False,False,...,-1,1470.55007618526,9.57750888570686e-255,1.75098878080951e-28,-352.441507967353,1,1,1,0.036950,-0.083103
1478,tmp,B,fe_exp_reviews_reviews,-1.193309,3.944105,-0.302555,7.622535e-01,-0.00426440753458848,0.00068360771342525,-6.23809159967119,...,-1,0.309792452704997,0.577856108473467,2.8353034355311,-351.778482684411,0.242282296608164,1,3,0.007549,-0.116170


In [10]:
sum(result_df['filename'].value_counts() == 1)

0

In [11]:
sum(result_df['filename'].value_counts() == 2)

7

In [12]:
sum(result_df['filename'].value_counts() == 3)

6

In [13]:
sum(result_df['filename'].value_counts() == 4)

362

Only getting those brands who have values for all the four models

In [14]:
count_list = result_df['filename'].value_counts()
valid_brands = count_list[count_list == 4].index.to_list() # having the values of all the four models
len(valid_brands)

362

In [15]:
result_df = result_df[result_df['filename'].isin(valid_brands)]
result_df

,tmp,filename,type,X_Estimate,X_Std. Error,X_t value,X_Pr(>|t|),IV_firststage_reviews_tw_Estimate,IV_firststage_reviews_tw_Std. Error,IV_firststage_reviews_tw_t value,...,HausWutest_Df,HausWutest_F,HausWutest_Pr(>F),Sargan_result_rsq,Sargan_result_adjrsq,Sargan_pvalue_rsq,Sargan_pvalue_adjrsq,num_sig_variables,Y_r.squared,Y_adj.r.squared
0,tmp,Brooks Brothers,fe_reviews_reviews,0.069505,0.019034,3.651686,2.663556e-04,-0.000100211019005055,0.000190635514929389,-0.52566815287368,...,-1,2.34354613255589,0.125940823615684,4.23449349278573,-353.500987164209,0.120362561387644,1,2,0.671032,0.625286
1,tmp,Brooks Brothers,fe_reviews_visits,0.068299,0.003906,17.487215,2.233359e-64,False,False,False,...,-1,1.9329373522152,0.164572372068733,1.21490380085773e-28,-355.731304347826,1,1,1,0.671424,0.625732
2,tmp,Brooks Brothers,fe_exp_reviews_reviews,1.541672,0.234746,6.567409,6.312742e-11,0.377923078481934,0.318135845031044,1.18792988713691,...,-1,14.7189627428136,0.000128155117934805,0.74492433429816,-357.468891520661,0.689035719250415,1,2,0.727018,0.689058
3,tmp,Brooks Brothers,fe_exp_reviews_visits,1.064166,0.030081,35.376507,3.701041e-219,False,False,False,...,-1,37.0578552273081,1.34043757762807e-09,2.74781634341386e-28,-355.731304347826,1,1,1,0.735753,0.699007
4,tmp,Torrid,fe_reviews_reviews,0.060262,0.021028,2.865750,4.187351e-03,0.00498750665721031,0.00258175309132841,1.93182945106654,...,-1,14.496389999924,0.000143060081773808,3.21982428292259,-346.092335329809,0.199905176680324,1,0,0.109297,0.019754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,tmp,REI,fe_exp_reviews_visits,17.982587,11.619486,1.547623,1.218157e-01,False,False,False,...,-1,159.888003382137,9.11891570094413e-36,3.13423717663725e-28,-347.234455958549,1,1,0,0.011222,-0.091762
1476,tmp,B,fe_reviews_reviews,1.637189,1.288917,1.270205,2.041267e-01,-0.000207663876637634,0.000482982056227962,-0.429961887734437,...,-1,4.07898337207887,0.0435235342635979,9.96297734472914,-343.774750838493,0.00686383694371739,1,1,0.044036,-0.075134
1477,tmp,B,fe_reviews_visits,4.949395,1.881332,2.630792,8.569535e-03,False,False,False,...,-1,1470.55007618526,9.57750888570686e-255,1.75098878080951e-28,-352.441507967353,1,1,1,0.036950,-0.083103
1478,tmp,B,fe_exp_reviews_reviews,-1.193309,3.944105,-0.302555,7.622535e-01,-0.00426440753458848,0.00068360771342525,-6.23809159967119,...,-1,0.309792452704997,0.577856108473467,2.8353034355311,-351.778482684411,0.242282296608164,1,3,0.007549,-0.116170


Extract the significant neighboring brands whose p-values for all the models are significant (<0.05) i.e., X_Pr(>|t|)

In [16]:
def filter_brands_pvalue(brand_pvalue):
    pvalues = brand_pvalue.values
    
    if (pvalues[0] < 0.05) and (pvalues[1] < 0.05) and (pvalues[2] < 0.05) and (pvalues[3] < 0.05):
        return True
    else:
        return False

In [17]:
significant_brands = result_df.groupby('filename')['X_Pr(>|t|)'].apply(filter_brands_pvalue)
significant_brands_list = significant_brands [significant_brands == True].index.to_list()

In [18]:
significant_brands_list

['Aubuchon Hardware',
 'Bath & Body Works',
 'Best Friends Pet Care',
 'Boston Market',
 'Brooks Brothers',
 "Chico's",
 "Dave & Buster's",
 'Everything but Water',
 'Free People',
 'Gap',
 "Jordan's Furniture",
 'Nordstrom',
 'Olive Garden',
 'Primark',
 'Skechers',
 "Spencer's",
 'Tous Les Jours',
 'Town Fair Tire',
 'Wegmans Food Markets',
 'Windsor',
 'Zumiez',
 'lululemon athletica']

### Performing the calculations for WIC and WIBC

Firstly Calculating the Inverse and Inverse-Exp of the Time_mins

In [19]:
travel_time['Time_mins_inv'] = 1/travel_time['Time_mins']
travel_time['Time_mins_inv_exp'] = 1/np.exp(travel_time['Time_mins'])
travel_time = travel_time.rename(columns={'From_PLACEKEY': 'Focal_Stores', 'To_PLACEKEY': 'Neib_Stores'})
travel_time

,Focal_Stores,Neib_Stores,Time_mins,Time_mins_inv,Time_mins_inv_exp
0,zzw-224@62k-p96-s5z,zzw-223@62k-ns4-pn5,20,0.050000,2.061154e-09
1,zzw-224@62k-p96-s5z,zzy-222@62k-pd8-975,20,0.050000,2.061154e-09
2,zzw-224@62k-p96-s5z,237-222@62k-p8v-z4v,16,0.062500,1.125352e-07
3,zzw-224@62k-p96-s5z,222-222@62k-p8v-2p9,12,0.083333,6.144212e-06
4,zzw-224@62k-p96-s5z,229-222@62k-p76-d9z,14,0.071429,8.315287e-07
...,...,...,...,...,...
139637,zzy-22f@62j-shz-vs5,22c-222@62j-sgs-w49,29,0.034483,2.543666e-13
139638,zzy-22f@62j-shz-vs5,22k-222@62j-sjr-z9f,21,0.047619,7.582560e-10
139639,zzy-22f@62j-shz-vs5,228-222@62j-sj3-v75,14,0.071429,8.315287e-07
139640,zzy-22f@62j-shz-vs5,22g-222@62j-sxw-33q,19,0.052632,5.602796e-09


1. Extracting all the stores of the significant brands and focal store
2. Calculating the Average Visits of each store to be used as average sales
3. Dropping all the duplicate rows to have a clean dataframe

In [20]:
brands_visits_focal_sig_neib = brands_visits[(brands_visits['brand_visitation'].isin(significant_brands_list)) | (brands_visits['brand_visitation']==brand)]
brands_visits_focal_sig_neib.loc[:,'avg_visits'] = brands_visits_focal_sig_neib.groupby('PLACEKEY')['visits_by_day'].transform('mean')
brands_visits_focal_sig_neib = brands_visits_focal_sig_neib.drop(columns=['date', 'visits_by_day', 'spend_by_day'])
brands_visits_focal_sig_neib = brands_visits_focal_sig_neib.drop_duplicates().reset_index(drop=True)
brands_visits_focal_sig_neib

/tmp/ipykernel_139382/4234074472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brands_visits_focal_sig_neib.loc[:,'avg_visits'] = brands_visits_focal_sig_neib.groupby('PLACEKEY')['visits_by_day'].transform('mean')


,PLACEKEY,brand_visitation,lat,lon,brand_standard,avg_visits
0,223-222@62j-spj-c3q,Aubuchon Hardware,42.542588,-71.466303,aubuchon hardware,4.802198
1,222-222@62k-3vv-xyv,Aubuchon Hardware,41.697077,-70.587265,aubuchon hardware,2.442308
2,zzw-22b@62k-p76-gkz,Bath & Body Works,42.222454,-71.023319,bath & body works,22.508242
3,224-222@62j-shb-6ff,Town Fair Tire,42.365204,-71.180655,town fair tire,2.074176
4,222-222@62j-srm-vmk,Dave & Buster's,42.300860,-71.381633,dave & buster's,846.741784
...,...,...,...,...,...,...
229,224-224@62j-szt-rtv,Town Fair Tire,42.501292,-70.925302,town fair tire,1.684211
230,227-224@62j-rjp-z75,Bath & Body Works,42.744194,-71.157932,bath & body works,7.870968
231,226-222@62j-sz7-gp9,Bath & Body Works,42.399945,-71.067746,bath & body works,4.290323
232,zzw-226@62k-prs-dgk,Bath & Body Works,41.641408,-70.991033,bath & body works,28.677419


Extracting the time_mins with all the focal stores belonging to brand under consideration, and all of the significant brands

In [24]:
focal_stores = brands_visits_focal_sig_neib[brands_visits_focal_sig_neib['brand_visitation'] == brand]['PLACEKEY'].unique().tolist()
sig_neib_stores = brands_visits_focal_sig_neib[brands_visits_focal_sig_neib['brand_visitation'].isin(significant_brands_list)]['PLACEKEY'].unique().tolist()
travel_time_filtered = travel_time[(travel_time['Focal_Stores'].isin(focal_stores)) & (travel_time['Neib_Stores'].isin(sig_neib_stores))]
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins,Time_mins_inv,Time_mins_inv_exp
615,zzw-22m@629-2rt-cyv,zzw-222@629-2fq-qvf,16,0.062500,1.125352e-07
616,zzw-22m@629-2rt-cyv,zzw-229@629-2rt-fzz,4,0.250000,1.831564e-02
635,zzw-22m@629-2rt-cyv,zzw-223@629-22b-3h5,21,0.047619,7.582560e-10
798,zzw-22m@629-2rt-cyv,222-222@629-2rt-cqz,4,0.250000,1.831564e-02
804,zzw-22m@629-2rt-cyv,224-222@629-2pw-zvf,22,0.045455,2.789468e-10
...,...,...,...,...,...
135484,zzw-22p@62j-srj-ffz,zzw-22s@62j-srm-vs5,1,1.000000,3.678794e-01
135500,zzw-22p@62j-srj-ffz,zzy-222@62j-srm-7nq,4,0.250000,1.831564e-02
135566,zzw-22p@62j-srj-ffz,222-222@62j-sr5-92k,16,0.062500,1.125352e-07
135576,zzw-22p@62j-srj-ffz,222-222@62j-srm-6ff,5,0.200000,6.737947e-03


Merging by Focal Stores to get the average visits of the focal stores

In [25]:
travel_time_filtered = pd.merge(left=travel_time_filtered, right=brands_visits_focal_sig_neib, how='left', left_on='Focal_Stores', right_on='PLACEKEY')
travel_time_filtered = travel_time_filtered.drop(columns=['PLACEKEY', 'brand_standard']).rename(columns={'brand_visitation':'Foc_Brand', 'lat':'Foc_lat', 'lon':'Foc_lon', 'avg_visits': 'Foc_avg_visits'})
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins,Time_mins_inv,Time_mins_inv_exp,Foc_Brand,Foc_lat,Foc_lon,Foc_avg_visits
0,zzw-22m@629-2rt-cyv,zzw-222@629-2fq-qvf,16,0.062500,1.125352e-07,Sephora,42.168174,-72.641450,28.986264
1,zzw-22m@629-2rt-cyv,zzw-229@629-2rt-fzz,4,0.250000,1.831564e-02,Sephora,42.168174,-72.641450,28.986264
2,zzw-22m@629-2rt-cyv,zzw-223@629-22b-3h5,21,0.047619,7.582560e-10,Sephora,42.168174,-72.641450,28.986264
3,zzw-22m@629-2rt-cyv,222-222@629-2rt-cqz,4,0.250000,1.831564e-02,Sephora,42.168174,-72.641450,28.986264
4,zzw-22m@629-2rt-cyv,224-222@629-2pw-zvf,22,0.045455,2.789468e-10,Sephora,42.168174,-72.641450,28.986264
...,...,...,...,...,...,...,...,...,...
404,zzw-22p@62j-srj-ffz,zzw-22s@62j-srm-vs5,1,1.000000,3.678794e-01,Sephora,42.301064,-71.383703,788.653846
405,zzw-22p@62j-srj-ffz,zzy-222@62j-srm-7nq,4,0.250000,1.831564e-02,Sephora,42.301064,-71.383703,788.653846
406,zzw-22p@62j-srj-ffz,222-222@62j-sr5-92k,16,0.062500,1.125352e-07,Sephora,42.301064,-71.383703,788.653846
407,zzw-22p@62j-srj-ffz,222-222@62j-srm-6ff,5,0.200000,6.737947e-03,Sephora,42.301064,-71.383703,788.653846


Merging by Neighboring Stores to get the average visits of the neighboring stores

In [26]:
travel_time_filtered = pd.merge(left=travel_time_filtered, right=brands_visits_focal_sig_neib, how='left', left_on='Neib_Stores', right_on='PLACEKEY')
travel_time_filtered = travel_time_filtered.drop(columns=['PLACEKEY','lat', 'lon', 'brand_standard']).rename(columns={'brand_visitation':'Neib_Brand', 'avg_visits': 'Neib_avg_visits'})
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins,Time_mins_inv,Time_mins_inv_exp,Foc_Brand,Foc_lat,Foc_lon,Foc_avg_visits,Neib_Brand,Neib_avg_visits
0,zzw-22m@629-2rt-cyv,zzw-222@629-2fq-qvf,16,0.062500,1.125352e-07,Sephora,42.168174,-72.641450,28.986264,Town Fair Tire,10.032967
1,zzw-22m@629-2rt-cyv,zzw-229@629-2rt-fzz,4,0.250000,1.831564e-02,Sephora,42.168174,-72.641450,28.986264,Bath & Body Works,27.159341
2,zzw-22m@629-2rt-cyv,zzw-223@629-22b-3h5,21,0.047619,7.582560e-10,Sephora,42.168174,-72.641450,28.986264,Chico's,2.142857
3,zzw-22m@629-2rt-cyv,222-222@629-2rt-cqz,4,0.250000,1.831564e-02,Sephora,42.168174,-72.641450,28.986264,Zumiez,9.230769
4,zzw-22m@629-2rt-cyv,224-222@629-2pw-zvf,22,0.045455,2.789468e-10,Sephora,42.168174,-72.641450,28.986264,Town Fair Tire,2.758242
...,...,...,...,...,...,...,...,...,...,...,...
404,zzw-22p@62j-srj-ffz,zzw-22s@62j-srm-vs5,1,1.000000,3.678794e-01,Sephora,42.301064,-71.383703,788.653846,Windsor,788.653846
405,zzw-22p@62j-srj-ffz,zzy-222@62j-srm-7nq,4,0.250000,1.831564e-02,Sephora,42.301064,-71.383703,788.653846,Spencer's,788.653846
406,zzw-22p@62j-srj-ffz,222-222@62j-sr5-92k,16,0.062500,1.125352e-07,Sephora,42.301064,-71.383703,788.653846,Best Friends Pet Care,0.730263
407,zzw-22p@62j-srj-ffz,222-222@62j-srm-6ff,5,0.200000,6.737947e-03,Sephora,42.301064,-71.383703,788.653846,Town Fair Tire,1.788462


#### Performing WIC Calculations by taking each neighboring store

In [67]:
def centrality_calc(group_df):
    neib_brand = group_df.name
    test_brand = travel_time_filtered[travel_time_filtered['Neib_Brand'] == neib_brand]
    neib_result_df = result_df [result_df['filename'] == neib_brand]
    
    dist_l = test_brand['Time_mins_inv'].values
    dist2_l = test_brand['Time_mins_inv_exp'].values
    average_sales = test_brand['Neib_avg_visits'].values
    est_reviews = neib_result_df[neib_result_df['type'] == 'fe_reviews_reviews']['X_Estimate'].values[0]
    est_reviews_exp = neib_result_df[neib_result_df['type'] == 'fe_exp_reviews_reviews']['X_Estimate'].values[0]

    weighted_influence = np.sum(np.array(dist_l).dot(np.array(average_sales))*float(est_reviews))
    weighted_influence_exp = np.sum(np.array(dist2_l).dot(np.array(average_sales))*float(est_reviews_exp))
    influence = np.sum(np.array(dist_l)*float(est_reviews))
    weighted_no_influence = np.sum(np.array(dist_l).dot(np.array(average_sales)))
    no_influence = np.sum(np.array(dist_l))
    avg_visits = np.mean(average_sales)
    dist_inv = np.mean([1.0 / d for d in np.array(dist_l)])
    num_stores = test_brand['Neib_Stores'].nunique()
    
    return pd.Series({'weighted_influence': weighted_influence, 'weighted_influence_exp': weighted_influence_exp, 'influence': influence,
                      'weighted_no_influence': weighted_no_influence, 'no_influence': no_influence, 'avg_visits':avg_visits,
                      'dist_inv': dist_inv, 'num_stores': num_stores})

In [70]:
neib_centrality = travel_time_filtered.groupby('Neib_Brand')[['Time_mins_inv', 'Time_mins_inv_exp', 'Neib_avg_visits']].apply(centrality_calc).reset_index()
neib_centrality

,Neib_Brand,weighted_influence,weighted_influence_exp,influence,weighted_no_influence,no_influence,avg_visits,dist_inv,num_stores
0,Aubuchon Hardware,53.585547,0.005761,7.656263,2.767073,0.395357,6.983117,18.571429,6.0
1,Bath & Body Works,55.292782,85.497969,0.474653,731.119635,6.276197,83.886105,15.320755,15.0
2,Best Friends Pet Care,-10.778149,-0.279335,-1.229393,3.588040,0.409264,10.467815,13.600000,3.0
3,Boston Market,-75.093320,-0.110367,-4.604282,29.819521,1.828358,13.289050,17.821429,8.0
4,Brooks Brothers,22.983087,61.891628,0.176691,330.668541,2.542131,115.726246,15.625000,4.0
5,Chico's,24.239916,187.329573,5.417315,13.490164,3.014881,3.810096,8.875000,5.0
6,Dave & Buster's,17.281841,145.409329,0.070968,490.018897,2.012259,125.811266,13.600000,3.0
7,Everything but Water,13.102214,109.708893,0.048400,398.329643,1.471429,162.056231,7.200000,3.0
8,Free People,14.770280,106.272421,0.074398,417.466597,2.102796,66.226003,15.066667,4.0
9,Gap,21.521448,54.438985,0.901282,150.904333,6.319616,18.569752,15.291667,13.0


### Making the Figure D1 and D2

1. Then we need to plot their X_Estimate for each of the four models
2. Then we need to put stars at the neighboring brands as per the p-value for each model \
-> *** means p_value <= 0.01 \
-> ** means 0.01 < p_value <= 0.05 \
-> * means 0.05 < p_value <= 0.1

In [ ]:
def get_pvalue_stars(p_value):
    if p_value <= 0.01:
        return '***'
    elif 0.01 < p_value <= 0.05:
        return '**'
    elif 0.05 < p_value <= 0.1:
        return '*'
    else:
        return ''

In [ ]:
coeff_est_df = result_df[result_df['filename'].isin(significant_brands_list)]
coeff_est_df['X_pvalue_stars'] = coeff_est_df['X_Pr(>|t|)'].apply(get_pvalue_stars)
coeff_est_df

In [ ]:
lin_reviews = coeff_est_df[coeff_est_df['type'] == 'fe_reviews_reviews'].sort_values('X_Estimate', ascending=False)
lin_visits = coeff_est_df[coeff_est_df['type'] == 'fe_reviews_visits'].sort_values('X_Estimate', ascending=False)
exp_reviews = coeff_est_df[coeff_est_df['type'] == 'fe_exp_reviews_reviews'].sort_values('X_Estimate', ascending=False)
exp_visits = coeff_est_df[coeff_est_df['type'] == 'fe_exp_reviews_visits'].sort_values('X_Estimate', ascending=False)

In [ ]:
def prepare_first_stage_results_review(row):
    # Twitter Vaues
    tw_estimate = format(float(row['IV_firststage_reviews_tw_Estimate']), '.2e')
    tw_std_err = format(float(row['IV_firststage_reviews_tw_Std. Error']), '.2e')
    tw_p_value = float(row['IV_firststage_reviews_tw_Pr(>|t|)'])
    
    # Facebook Vaues
    fb_estimate = format(float(row['IV_firststage_reviews_fb_Estimate']), '.2e')
    fb_std_err = format(float(row['IV_firststage_reviews_fb_Std. Error']), '.2e')
    fb_p_value = float(row['IV_firststage_reviews_fb_Pr(>|t|)'])
    
    # Instagram Vaues
    ig_estimate = format(float(row['IV_firststage_reviews_ig_Estimate']), '.2e')
    ig_std_err = format(float(row['IV_firststage_reviews_ig_Std. Error']), '.2e')
    ig_p_value = float(row['IV_firststage_reviews_ig_Pr(>|t|)'])
    
    # Getting table entries
    twitter_likes = str(tw_estimate) + get_pvalue_stars(tw_p_value) + "({})".format(tw_std_err)
    facebook_likes = str(fb_estimate) + get_pvalue_stars(fb_p_value) + "({})".format(fb_std_err)
    instagram_likes = str(ig_estimate) + get_pvalue_stars(ig_p_value) + "({})".format(ig_std_err)
    
    # Calculating significant variables
    num_sig = 0
    
    if (tw_p_value <= 0.05):
        num_sig += 1

    if (fb_p_value <= 0.05):
        num_sig += 1

    if (ig_p_value <= 0.05):
        num_sig += 1
    
    # Getting WaldTest F-Stats
    wald_f_stat = row['WaldTest_F']
        
    return pd.Series([twitter_likes, facebook_likes, instagram_likes, num_sig, wald_f_stat],
                     index=['Twitter Likes', 'Facebook Likes', 'Instagram Likes', 'Num Sig', 'Weka Instrument (F-stats)'])

In [ ]:
def prepare_first_stage_results_visits(row):
    # Visits Vaues
    visits_estimate = format(float(row['IV_firststage_visits_Estimate']), '.2e')
    visits_std_err = format(float(row['IV_firststage_visits_Std. Error']), '.2e')
    visits_p_value = float(row['IV_firststage_visits_Pr(>|t|)'])
        
    # Getting table entries
    visits = str(visits_estimate) + get_pvalue_stars(visits_p_value) + "({})".format(visits_std_err)
        
    # Getting WaldTest F-Stats
    wald_f_stat = row['WaldTest_F']
        
    return pd.Series([visits, wald_f_stat],
                     index=['Visits', 'Weka Instrument (F-stats)'])

In [ ]:
first_stage_linear_reviews = lin_reviews.apply(prepare_first_stage_results_review, axis=1, result_type='expand').reset_index(drop=True)
first_stage_linear_reviews

In [ ]:
first_stage_linear_visits = lin_visits.apply(prepare_first_stage_results_visits, axis=1, result_type='expand').reset_index(drop=True)
first_stage_linear_visits

In [ ]:
first_stage_exp_reviews = exp_reviews.apply(prepare_first_stage_results_review, axis=1, result_type='expand').reset_index(drop=True)
first_stage_exp_reviews

In [ ]:
first_stage_exp_visits = exp_visits.apply(prepare_first_stage_results_visits, axis=1, result_type='expand').reset_index(drop=True)
first_stage_exp_visits

In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
lin_reviews['Lower Bound'] = lin_reviews['X_Estimate'] - 1.96 * lin_reviews['X_Std. Error']
lin_reviews['Upper Bound'] = lin_reviews['X_Estimate'] + 1.96 * lin_reviews['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in lin_reviews.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xlabel('Estimate')
ax.set_title('Linear Reviews')
ax.set_xscale('symlog')

# Show the plot
plt.show()

In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
lin_visits['Lower Bound'] = lin_visits['X_Estimate'] - 1.96 * lin_visits['X_Std. Error']
lin_visits['Upper Bound'] = lin_visits['X_Estimate'] + 1.96 * lin_visits['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in lin_visits.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xlabel('Estimate')
ax.set_title('Linear Visits')
ax.set_xscale('symlog')
ax.set_xlim()

# Show the plot
plt.show()

In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
exp_reviews['Lower Bound'] = exp_reviews['X_Estimate'] - 1.96 * exp_reviews['X_Std. Error']
exp_reviews['Upper Bound'] = exp_reviews['X_Estimate'] + 1.96 * exp_reviews['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in exp_reviews.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xlabel('Estimate')
ax.set_title('Exponential Reviews')
ax.set_xscale('symlog')

# Show the plot
plt.show()

In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
exp_visits['Lower Bound'] = exp_visits['X_Estimate'] - 1.96 * exp_visits['X_Std. Error']
exp_visits['Upper Bound'] = exp_visits['X_Estimate'] + 1.96 * exp_visits['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in exp_visits.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xlabel('Estimate')
ax.set_title('Exponential Visits')
ax.set_xscale('symlog')

# Show the plot
plt.show()